## PyTorch Excercises

In [14]:
import torch
import numpy as np

In [2]:
print(torch.__version__)

2.5.1


In [4]:
print(torch.backends.mps.is_available()) # for mac silicon chip(M1, M2...)

True


In [6]:
tensor0d = torch.tensor(1)  #scalar tensor with rank 0
tensor1d = torch.tensor([1,2,3])
tensor2d = torch.tensor([[1,2],
                         [3,4]])
tensor3d = torch.tensor([[[1,2], [3,4]],
                         [[5,6], [7,8]]])

In [7]:
print(tensor3d)

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [12]:
print(tensor3d.dtype)

torch.int64


In [17]:
aray3d = np.array([[[1, 2], [3, 4]], 
                  [[5, 6], [7, 8]]])

print(aray3d)

[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]


In [18]:
print(aray3d.dtype)

int64


In [19]:
tensor3d_2 = torch.tensor(aray3d)
tensor3d_3 = torch.from_numpy(aray3d)

aray3d[0,0,0] = 999

print(tensor3d_2)  # remains unchaged


tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [20]:
print(tensor3d_3) #changed, because of memory sharing

tensor([[[999,   2],
         [  3,   4]],

        [[  5,   6],
         [  7,   8]]])


In [27]:
tensor2d = torch.tensor([[1,2,3],
                        [4,5,6]])
print(tensor2d.shape)

torch.Size([2, 3])


In [26]:
print(tensor2d.reshape(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [28]:
print(tensor2d.view(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [29]:
print(tensor2d.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [30]:
print(tensor2d.matmul(tensor2d.T))

tensor([[14, 32],
        [32, 77]])


In [31]:
print(tensor2d @ tensor2d.T) # does the same but more compactly

tensor([[14, 32],
        [32, 77]])


## Logistics Regression Forward Pass

In [35]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2] , requires_grad = True)
b = torch.tensor([0.0] ,requires_grad = True)
z = x1 * w1 + b

print('z',z)

a = torch.sigmoid(z)
print('a', a)

loss = F.binary_cross_entropy(a,y)
print('loss', loss)

loss.backward()

print(w1.grad)
print(b.grad)

z tensor([2.4200], grad_fn=<AddBackward0>)
a tensor([0.9183], grad_fn=<SigmoidBackward0>)
loss tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([-0.0898])
tensor([-0.0817])


## Implemention Multi Layer Neural Network

In [36]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            #1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            #2nd Hidden Layer
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

        

In [38]:
model = NeuralNetwork(50 , 3)

In [39]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [40]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

2213


In [41]:
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0638, -0.1205, -0.0073,  ..., -0.0409,  0.1127, -0.0998],
        [ 0.0574,  0.0846, -0.0344,  ...,  0.1202, -0.0602,  0.0046],
        [ 0.0269,  0.0763,  0.1246,  ...,  0.0533,  0.1044,  0.0038],
        ...,
        [-0.1143, -0.0622, -0.0645,  ..., -0.0420,  0.0669, -0.1327],
        [ 0.1154,  0.1063,  0.0758,  ..., -0.0080, -0.0946,  0.0336],
        [ 0.0768,  0.0786,  0.0756,  ...,  0.0845, -0.1243, -0.1155]],
       requires_grad=True)


In [42]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [44]:
print(model.layers[0].bias.shape)

torch.Size([30])


In [45]:
torch.manual_seed(123)

X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.0835,  0.0365,  0.0126]], grad_fn=<AddmmBackward0>)


In [46]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.0835,  0.0365,  0.0126]])


In [47]:
with torch.no_grad():
    out = torch.softmax(model(X), dim = 1)
print(out)

tensor([[0.3098, 0.3492, 0.3410]])


## Setting up Dataloaders

In [48]:
 # toy dataset
X_train = torch.tensor([[-1.2, 3.1],
           [-0.9, 2.9],
           [-0.5, 2.6],
           [2.3, -1.1],
           [2.7, -1.5]])

y_train = torch.tensor([0 , 0, 0, 1,1])
 X_test = torch.tensor([
           [-0.8, 2.8],
           [2.6, -1.6],
       ])
y_test = torch.tensor([0, 1])

In [64]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X,y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_df = ToyDataset(X_train, y_train)
test_df = ToyDataset(X_test, y_test)

In [59]:
print(len(train_df))

5


In [65]:
# instantiating data loaders

from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_df,
    batch_size = 2,
    shuffle = True,
    num_workers = 0
)

In [66]:
test_loader = DataLoader(
    dataset = test_df,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [69]:
for idx, (x,y) in enumerate(train_loader):
     print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-0.9000,  2.9000],
        [ 2.3000, -1.1000]]) tensor([0, 1])
Batch 2: tensor([[ 2.7000, -1.5000],
        [-0.5000,  2.6000]]) tensor([1, 0])
Batch 3: tensor([[-1.2000,  3.1000]]) tensor([0])


In [71]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True   # drop last batch if not enought data to match batch size
)

In [72]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [ 2.7000, -1.5000]]) tensor([1, 1])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.9000,  2.9000]]) tensor([0, 0])


## Neural Network Training in Pytorch

In [74]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs = 2, num_outputs = 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features,labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels) # loss function

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval() #optional model evaluation
        

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


In [75]:
# predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [76]:
# probabilities
torch.set_printoptions(sci_mode = False)
probas = torch.softmax(outputs, dim = 1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [77]:
predictions = torch.argmax(probas, dim = 1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [78]:
predictions = torch.argmax(outputs, dim = 1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [79]:
predictions == y_train

tensor([True, True, True, True, True])

In [81]:
torch.sum(predictions == y_train)

tensor(5)

In [83]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim = 1)
        compare = labels == predictions
        correct +=torch.sum(compare)
        total_examples  +=len(compare)

    return (correct /total_examples) .item()

In [84]:
print(compute_accuracy(model,train_loader))

1.0


In [85]:
print(compute_accuracy(model,test_loader))

1.0


## Saving and loading models

In [88]:
torch.save(model.state_dict(), "model.pth")

In [89]:
# the NeuralNetwork(2, 2) architecture needs to match the original saved model exactly
model = NeuralNetwork(2,2) 
model.load_state_dict(torch.load("model.pth"))

/var/folders/xy/l4m6vnmx3dv0p46qkb2h9s9m0000gn/T/ipykernel_34022/1839425069.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth")

<All keys matched successfully>